# Eggs: reduced-form pass-through diagnostics

This notebook is a step-by-step diagnostic for a rockets-and-feathers phenomenon in eggs.
It is intentionally simple: one regression at a time, with explicit interpretation after each step.

In [26]:
# Imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.graph_objects as go
from linearmodels.iv import IV2SLS
from scipy.stats import chi2
from statsmodels.regression.linear_model import OLS
from scipy import stats

DATA_FOLDER = '/home/akimovh/rockets_feathers/eggs/data/usda/'
PLOT_FOLDER = '/home/akimovh/rockets_feathers/eggs/plots/usda/'




We begin by downloading the data. All data are collected from public sources; see the documentation for details on data collection.

In [6]:
# -----------------------
# Load data + prep
# -----------------------
df = pd.read_csv(DATA_FOLDER + "reduced_form.csv")
retail_df = pd.read_csv(DATA_FOLDER + "retail_price.csv")

df = pd.merge(df, retail_df, on="date", how="inner")
df["date"] = pd.to_datetime(df["date"])

# Feed-cost proxy
df["cost_feed_us"] = 0.75 * df["cost_corn_us"] + 0.25 * df["cost_sb_us"] # Created in first iterations, this variable has low predictive power, so it's not used futher.

# Pass-through and adjustment dynamics in the U.S. egg market (reduced-form diagnostics)

This notebook builds a **minimal, transparent set of reduced-form checks** before moving to an IV/ECM design. (CHANGE)

**Data (monthly):**
- **Wholesale / upstream price (`wh`)**: USDA weekly egg report aggregated to month (cents/dozen)
- **Retail price (`rt`)**: BLS retail series (cents/dozen)
- **Supply shock proxy (`death`)**: hen losses (rendered deaths), monthly flow

**Roadmap**
1. **Regression 1 (relevance of shifters):** Do plausibly exogenous supply shocks predict **wholesale price changes**?
2. **Regression 2 (pass-through + timing diagnostics):** Does retail respond to wholesale, and how fast? (Include a lead placebo.)
3. **Regression 2b (asymmetry in speed):** Are responses to wholesale increases vs decreases different by horizon?
4. **Regression 3 (next step):** IV pass-through instrumenting short-run wholesale movements with avian-flu mortality windows (and their lags).


## 1) Wholesale price prediction / instrument relevance

**Question.** Do plausibly exogenous supply shocks move the upstream egg price in a strong, predictable way?

**Why this step?** If mortality shocks do *not* predict upstream price movements, they are not useful for IV later.

**Specification choices**
- Work in **monthly changes** (first differences): we want short-run movements and to avoid spurious trend relationships (feed and eggs costs might be trending parallel).
- Control for **month-of-year fixed effects** to absorb seasonality (Easter/holiday baking, etc.).
- Use **HAC** standard errors (monthly data + distributed lags ⇒ serial correlation likely).

**Mortality windows (biological timing)**
The production chain has adjustment delays: flock losses today can tighten supply immediately, but replacement takes months.
We summarize mortality into three windows (in millions of hens):
- `D_0_2`: contemporaneous + 1–2 month lag (near-term tightening)
- `D_3_6`: 3–6 month lag (medium-term)
- `D_7_12`: 7–12 month lag (longer-term recovery window)

(Feed costs are a natural shifter, but in this dataset they have weaker predictive power for upstream changes; we keep them as an optional robustness control.)


In [7]:
# -----------------------
# Build analysis df
# -----------------------
assym_df = df[["date", "cost_feed_us", "price_large", "loss_dth_render"]].copy()
assym_df.columns = ["date", "cost", "price", "death"]
assym_df = assym_df.sort_values("date")


# Outcomes / shifters
assym_df["d_price"]  = assym_df["price"].diff()           # cents/dozen
assym_df["death_m"]  = assym_df["death"] / 1_000_000.0    # million hens (flow/shock)

# Creating death variables

for l in range(0, 13):
    assym_df[f"death_m_lag{l}"] = assym_df["death_m"].shift(l)

assym_df["D_0_2"]  = assym_df[[f"death_m_lag{l}" for l in range(0, 3)]].sum(axis=1)
assym_df["D_3_6"]  = assym_df[[f"death_m_lag{l}" for l in range(3, 7)]].sum(axis=1)
assym_df["D_7_12"] = assym_df[[f"death_m_lag{l}" for l in range(7, 13)]].sum(axis=1)

# -----------------------
# Seasonality controls (month-of-year FE)
# -----------------------
assym_df["month"] = assym_df["date"].dt.month
month_dummies = pd.get_dummies(assym_df["month"], prefix="m", drop_first=True).astype(int)
assym_df = pd.concat([assym_df, month_dummies], axis=1)

# Drop NA created by differencing / lags
assym_df = assym_df.dropna().copy()


In [8]:
# -----------------------
# Regression 1: Δ wholesale on mortality windows (and month FE)
# -----------------------
y = assym_df["d_price"]

# Baseline: mortality windows only (plus month FE)
X_cols = ["D_0_2", "D_3_6", "D_7_12"] + list(month_dummies.columns)

# Optional robustness: add feed-cost changes (often weak here)
# X_cols = (["D_0_2", "D_3_6", "D_7_12", "d_cost10"]
#          + [f"d_cost10_lag{l}" for l in range(1, K_feed + 1)]
#          + list(month_dummies.columns))

X = sm.add_constant(assym_df[X_cols])

m1 = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": 3})
print(m1.summary())

                            OLS Regression Results                            
Dep. Variable:                d_price   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     5.113
Date:                Wed, 14 Jan 2026   Prob (F-statistic):           2.28e-06
Time:                        11:58:15   Log-Likelihood:                -449.12
No. Observations:                  81   AIC:                             928.2
Df Residuals:                      66   BIC:                             964.1
Df Model:                          14                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -35.9924    113.521     -0.317      0.7

In [9]:
# -----------------------
# Joint tests (block relevance)
# -----------------------
def joint_f_test(m1, X, cols):
    R = np.zeros((len(cols), X.shape[1]))
    for i, c in enumerate(cols):
        R[i, X.columns.get_loc(c)] = 1.0
    return m1.f_test(R)

# feed_terms  = ["d_cost10"] + [f"d_cost10_lag{l}" for l in range(1, K_feed + 1)]
death_terms = ["D_0_2", "D_3_6", "D_7_12"]
# all_terms   = death_terms

print("\n--- Joint tests (H0: block = 0) ---")
# print("Feed block:",  joint_f_test(model, X, feed_terms))
print("Death block:", joint_f_test(m1, X, death_terms))
# print("All shifters:", joint_f_test(model, X, all_terms))



--- Joint tests (H0: block = 0) ---
Death block: <F test: F=4.342112172898348, p=0.00746412415738067, df_denom=66, df_num=3>


**Interpretation (Regression 1).**
- The mortality windows have economically and statistically meaningful predictive power for upstream price changes (joint test).
- Feed-cost changes are comparatively weak in this reduced-form (optional robustness rather than baseline (We exluded feed costs from final view of Regression 1)).

**Implication for later.** Mortality windows look promising as *excluded shifters* for IV pass-through.


## 2) Retail response to wholesale changes (pass-through + timing)

**Question.** Does retail price change when wholesale price changes, and how quickly?

**Why differences?** Retail and wholesale levels are trending and highly persistent; the short-run adjustment dynamics are most transparently studied in **changes**.

**Distributed-lag model (symmetric).**
We regress monthly retail changes on contemporaneous and lagged wholesale changes:
$$
\Delta p^R_t = \alpha + \sum_{k=0}^{K} \beta_k \Delta p^W_{t-k} + \text{month FE} + u_t.
$$
The cumulative pass-through by horizon is $\sum_{k=0}^h \beta_k$.

**Timing / endogeneity diagnostic (lead placebo).**
We also include a small number of *leads* of wholesale changes (e.g., $\Delta p^W_{t+1}, \Delta p^W_{t+2}$).
If leads predict $\Delta p^R_t$, that suggests either:
- dating/aggregation mismatch (monthly averaging), and/or
- omitted persistent shocks common to both series.

This diagnostic motivates IV in Regression 3.


In [10]:
# -----------------------
# Build analysis df
# -----------------------
assym_df = df[["date", "price_large", 'retail_price']].copy()
assym_df.columns = ["date", "wh", "rt"]
assym_df = assym_df.sort_values("date")

assym_df["d_rt"] = assym_df["rt"].diff()     # Δ retail (cents/doz)
assym_df["d_wh"] = assym_df["wh"].diff()     # Δ wholesale (cents/doz)

K = 9     # number of wholesale lags
L = 4     # number of wholesale leads (placebo test)

# Leads and lags of Δ wholesale (note: negative lags are leads because shift(-1)=t+1)
for l in range(-L, K + 1):
    if l == 0:
        continue
    assym_df[f"d_wh_lag{l}"] = assym_df["d_wh"].shift(l)

# Month FE
assym_df["month"] = assym_df["date"].dt.month
month_dummies = pd.get_dummies(assym_df["month"], prefix="m", drop_first=True).astype(int)
assym_df = pd.concat([assym_df, month_dummies], axis=1)

# Drop NA created by diffs/lags/leads
assym_df = assym_df.dropna().copy()

### Visual check
We start with a simple plot of retail vs wholesale levels to confirm co-movement and identify obvious breaks.


In [11]:
x = assym_df.date

# UC Berkeley colors
berkeley_blue = "#002676"
california_gold = "#FDB515"

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=assym_df.rt,
    mode="lines",
    name="Retail price",
    line=dict(color=berkeley_blue, width=3.5)
))

fig.add_trace(go.Scatter(
    x=x,
    y=assym_df.wh,
    mode="lines",
    name="Wholesale price",
    line=dict(color=california_gold, width=3.5)
))

fig.update_layout(
    title="Retail and wholesale price of eggs",
    xaxis_title="Date",
    yaxis_title="Price(cents/dozen)",

    # ECONOMETRICA / QJE STYLE
    font=dict(
        family="Times New Roman",
        size=14,
        color="black"
    ),
    template="simple_white",

    # Axes styling
    xaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        linecolor="black",
        linewidth=1,
        mirror=True,
        ticks="outside",
        tickwidth=1,
        tickcolor="black"
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        linecolor="black",
        linewidth=1,
        mirror=True,
        ticks="outside",
        tickwidth=1,
        tickcolor="black"
    ),

    # Legend OUTSIDE (right)
    legend=dict(
        x=1.02,
        y=1,
        xanchor="left",
        yanchor="top",
        font=dict(size=15),
        bgcolor="rgba(255,255,255,0)",
        bordercolor="rgba(0,0,0,0)"
    ),

    margin=dict(r=150),
    width=850,
    height=520
)

fig.show()


The series strongly co-move, especially during 2022–2025. Retail is smoother (consistent with menu costs/contracts/averaging).
This motivates estimating a distributed-lag pass-through model in changes.

In [12]:
y = assym_df["d_rt"]

X_cols = (
    ["d_wh"] +
    [f"d_wh_lag{l}" for l in range(-L, K + 1) if l != 0] +
    list(month_dummies.columns)
)

X = sm.add_constant(assym_df[X_cols])

m2 = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": 12})
print(m2.summary())


                            OLS Regression Results                            
Dep. Variable:                   d_rt   R-squared:                       0.915
Model:                            OLS   Adj. R-squared:                  0.875
Method:                 Least Squares   F-statistic:                     551.5
Date:                Wed, 14 Jan 2026   Prob (F-statistic):           1.57e-55
Time:                        11:58:20   Log-Likelihood:                -289.77
No. Observations:                  79   AIC:                             631.5
Df Residuals:                      53   BIC:                             693.1
Df Model:                          25                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.2142      3.205      1.939      0.0

In [13]:
beta_cols = ["d_wh"] + [f"d_wh_lag{l}" for l in range(1, K + 1)]
lead_cols = [f"d_wh_lag{l}" for l in range(-L, 0)]

def joint_f_test(model, X, cols):
    R = np.zeros((len(cols), X.shape[1]))
    for i, c in enumerate(cols):
        R[i, X.columns.get_loc(c)] = 1.0
    return model.f_test(R)

print("\nWholesale lag block joint test (H0: all lags 0..K = 0):")
print(joint_f_test(m2, X, beta_cols))

print("\nWholesale lead placebo joint test (H0: all leads = 0):")
print(joint_f_test(m2, X, lead_cols))

cum_pass = float(m2.params[beta_cols].sum())
print("\nCumulative pass-through (0..K):", cum_pass)

cum_path = m2.params[beta_cols].cumsum()
print("\nCumulative by lag:")
print(cum_path)



Wholesale lag block joint test (H0: all lags 0..K = 0):
<F test: F=412.4619167339383, p=1.532556001248458e-46, df_denom=53, df_num=10>

Wholesale lead placebo joint test (H0: all leads = 0):
<F test: F=6.897466783860566, p=0.000151358012958123, df_denom=53, df_num=4>

Cumulative pass-through (0..K): 0.8659619204522787

Cumulative by lag:
d_wh         0.226207
d_wh_lag1    0.617457
d_wh_lag2    0.689202
d_wh_lag3    0.804736
d_wh_lag4    0.786349
d_wh_lag5    0.839930
d_wh_lag6    0.774869
d_wh_lag7    0.846333
d_wh_lag8    0.840090
d_wh_lag9    0.865962
dtype: float64


**Interpretation (Regression 2).**
- The wholesale distributed-lag block is strongly significant and cumulative pass-through is economically large.
- The lead placebo can be statistically significant even when small in magnitude; this is a warning sign for clean causal interpretation of OLS dynamics (Need instruments for 3 lags)
- We proceed with this OLS as a **diagnostic** and use IV later for causal claims.


### Split wholesale changes into positive and negative (asymmetric dynamics)

To test “rockets & feathers”-style asymmetry, we split wholesale changes:
- `d_wh_pos = max(d_wh, 0)`
- `d_wh_neg = min(d_wh, 0)` (non-positive)

Then estimate a symmetric distributed-lag model with separate coefficients for positive vs negative wholesale changes and compute **cumulative differences by horizon**:
$$
\sum_{k=0}^h \beta^{+}_k - \sum_{k=0}^h \beta^{-}_k.
$$

This tells us whether retail adjusts faster to cost increases than to cost decreases at short horizons (speed asymmetry).


In [14]:
# -----------------------
# Build analysis df
# -----------------------

assym_df = df[["date", "price_large", "retail_price"]].copy()
assym_df.columns = ["date", "wh", "rt"]
assym_df = assym_df.sort_values("date")

assym_df["d_rt"] = assym_df["rt"].diff()
assym_df["d_wh"] = assym_df["wh"].diff()

# Split wholesale changes
assym_df["d_wh_pos"] = assym_df["d_wh"].clip(lower=0)
assym_df["d_wh_neg"] = assym_df["d_wh"].clip(upper=0)   # ≤ 0

K = 9  # horizon for asymmetric dynamics (kept short given sample size)

# Lags for + and -
for l in range(1, K + 1):
    assym_df[f"d_wh_pos_lag{l}"] = assym_df["d_wh_pos"].shift(l)
    assym_df[f"d_wh_neg_lag{l}"] = assym_df["d_wh_neg"].shift(l)

# Month FE
assym_df["month"] = assym_df["date"].dt.month
month_dummies = pd.get_dummies(assym_df["month"], prefix="m", drop_first=True).astype(int)
assym_df = pd.concat([assym_df, month_dummies], axis=1)

assym_df = assym_df.dropna().copy()


In [15]:
# -----------------------
# Regression 2b: asymmetric pass-through (Δ wholesale + vs -)
# -----------------------

y = assym_df["d_rt"]

pos_cols = ["d_wh_pos"] + [f"d_wh_pos_lag{l}" for l in range(1, K + 1)]
neg_cols = ["d_wh_neg"] + [f"d_wh_neg_lag{l}" for l in range(1, K + 1)]

X_cols = pos_cols + neg_cols + list(month_dummies.columns)
X = sm.add_constant(assym_df[X_cols])

m2b = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": K})
print(m2b.summary())

                            OLS Regression Results                            
Dep. Variable:                   d_rt   R-squared:                       0.922
Model:                            OLS   Adj. R-squared:                  0.875
Method:                 Least Squares   F-statistic:                     610.0
Date:                Wed, 14 Jan 2026   Prob (F-statistic):           8.54e-56
Time:                        11:58:21   Log-Likelihood:                -302.14
No. Observations:                  83   AIC:                             668.3
Df Residuals:                      51   BIC:                             745.7
Df Model:                          31                                         
Covariance Type:                  HAC                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            11.7331      4.955      2.368

In [17]:
pos_cols = ["d_wh_pos"] + [f"d_wh_pos_lag{l}" for l in range(1, K+1)]
neg_cols = ["d_wh_neg"] + [f"d_wh_neg_lag{l}" for l in range(1, K+1)]

# cumulative differences by horizon
for h in range(0, K+1):
    pos_h = pos_cols[:h+1]
    neg_h = neg_cols[:h+1]

    w = np.zeros(X.shape[1])
    for c in pos_h:
        w[X.columns.get_loc(c)] += 1.0
    for c in neg_h:
        w[X.columns.get_loc(c)] -= 1.0

    test = m2b.wald_test(w, scalar=True)
    diff = float(m2b.params[pos_h].sum() - m2b.params[neg_h].sum())

    print(f"h={h:2d}  cum_diff={diff: .4f}   chi2={float(test.statistic): .3f}   p={float(test.pvalue): .3f}")


h= 0  cum_diff= 0.0324   chi2= 0.209   p= 0.648
h= 1  cum_diff= 0.0337   chi2= 0.241   p= 0.623
h= 2  cum_diff=-0.1334   chi2= 2.105   p= 0.147
h= 3  cum_diff=-0.2180   chi2= 7.201   p= 0.007
h= 4  cum_diff=-0.1181   chi2= 2.003   p= 0.157
h= 5  cum_diff=-0.1285   chi2= 1.534   p= 0.216
h= 6  cum_diff=-0.1522   chi2= 1.322   p= 0.250
h= 7  cum_diff=-0.1392   chi2= 2.458   p= 0.117
h= 8  cum_diff=-0.2727   chi2= 5.537   p= 0.019
h= 9  cum_diff=-0.0095   chi2= 0.041   p= 0.840


**Interpretation (Regression 2b).**
The horizon-by-horizon Wald tests show evidence consistent with **asymmetry in adjustment speed** (at some horizons),
even if the “full-horizon” cumulative difference is not always significant.

**Next step.** Because wholesale changes may reflect common shocks or timing mismatch (lead placebo),
we move to an IV design that isolates supply-driven wholesale variation using mortality windows.


## 3) Adding IV (Regression 3)

**Goal.** Estimate pass-through using only the component of wholesale price changes driven by plausibly exogenous supply shocks.

**Endogeneity concern.** Even if egg demand is “stable,” wholesale changes can still co-move with omitted shocks (and monthly averaging can create lead predictability).
So we avoid interpreting OLS dynamics as causal.

**IV design (baseline).**
- Treat the *short-run* wholesale changes as endogenous: $\Delta p^W_t, \Delta p^W_{t-1}, \Delta p^W_{t-2}, \Delta p^W_{t-3}$.
- Include longer lags (4–9) as controls.
- Use mortality windows as excluded instruments **and include their lags (0–3)** to strengthen the first stage for lagged wholesale changes.

We will evaluate:
- first-stage strength for each endogenous regressor,
- stability across alternative “endogenous block” choices (0 only vs 0–1 vs 0–2),
- implied cumulative pass-through.


In [18]:
# -----------------------
# Build analysis df
# -----------------------
assym_df = (
    df[["date", "price_large", "retail_price", "loss_dth_render"]]
    .rename(columns={"price_large": "wh", "retail_price": "rt", "loss_dth_render": "death"})
    .sort_values("date")
    .assign(death_m=lambda x: x["death"] / 1e6)
)

for lag in range(13):
    assym_df[f"death_m_lag{lag}"] = assym_df["death_m"].shift(lag)

assym_df["D_0_2"]  = assym_df[[f"death_m_lag{l}" for l in range(0, 3)]].sum(axis=1)
assym_df["D_3_6"]  = assym_df[[f"death_m_lag{l}" for l in range(3, 7)]].sum(axis=1)
assym_df["D_7_12"] = assym_df[[f"death_m_lag{l}" for l in range(7, 13)]].sum(axis=1)

for window in ["D_0_2", "D_3_6", "D_7_12"]:
    for lag in range(1, 4):
        assym_df[f"{window}_lag_{lag}"] = assym_df[window].shift(lag)


assym_df["d_rt"]  = assym_df["rt"].diff()           # cents/dozen
assym_df["d_wh"] = assym_df["wh"].diff()     # $10/ton change

# -----------------------
# Lags: feed (short) + death (long windows)
# -----------------------
K = 9
for l in range(1, K + 1):
    assym_df[f"d_wh_lag{l}"] = assym_df["d_wh"].shift(l)
    
# -----------------------
# Month FE
# -----------------------
assym_df["month"] = assym_df["date"].dt.month
month_dummies = pd.get_dummies(assym_df["month"], prefix="m", drop_first=True).astype(int)
assym_df = pd.concat([assym_df, month_dummies], axis=1)

# Drop NA created by diffs/lags
assym_df = assym_df.dropna().copy()


In [19]:
# --- Define variable blocks ---
endog_cols = ["d_wh"] + [f"d_wh_lag{l}" for l in range(1, 4)]
exog_cols  = [f"d_wh_lag{l}" for l in range(4, K + 1)] + list(month_dummies.columns)
iv_cols    = (
    ["D_0_2", "D_3_6", "D_7_12"]
    + [f"{w}_lag_{l}" for w in ["D_0_2", "D_3_6", "D_7_12"] for l in range(1, 4)]
)

y      = assym_df["d_rt"]
endog  = assym_df[endog_cols]
exog   = sm.add_constant(assym_df[exog_cols])
instruments = assym_df[iv_cols]

# --- Estimate ---
m3 = IV2SLS(y, exog, endog, instruments).fit(
    cov_type="kernel", kernel="bartlett", bandwidth=12
)

What we check next:

1. **Main IV estimates** (are the key wholesale coefficients sensible?).
2. **First-stage strength** for each endogenous regressor (weak IV would invalidate inference).
3. **Cumulative pass-through** and **speed of adjustment** (how much happens immediately vs. later).
4. **Specification diagnostics**: endogeneity tests (Wu–Hausman / Durbin) and overidentification tests (Sargan).
5. **Sensitivity**: results across alternative endogenous blocks and HAC bandwidth choices.


In [20]:
key_terms = ["d_wh"] + [f"d_wh_lag{l}" for l in range(1, 4)]

# Point estimates + SEs just for key terms
out = pd.DataFrame({
    "beta": m3.params.reindex(key_terms),
    "se":   m3.std_errors.reindex(key_terms),
    "t":    (m3.params.reindex(key_terms) / m3.std_errors.reindex(key_terms)),
    "p":    2 * (1 - stats.norm.cdf(np.abs(m3.params.reindex(key_terms) / m3.std_errors.reindex(key_terms))))
})

print("Main IV estimates")
print(out)

print()

print("First-stage strength")
print(m3.first_stage.diagnostics[['rsquared',  'partial.rsquared',  'shea.rsquared',      'f.stat',   'f.pval']])

print()

print("Cumulative pass-through")
betas = m3.params.reindex(key_terms)
cum0 = betas["d_wh"]
cum1 = betas[["d_wh","d_wh_lag1"]].sum()
cum2 = betas[["d_wh","d_wh_lag1","d_wh_lag2"]].sum()
cum3 = betas[key_terms].sum()

print(pd.Series({"cum0":cum0, "cum1":cum1, "cum2":cum2, "cum3":cum3}))

Main IV estimates
               beta        se          t             p
d_wh       0.242541  0.038128   6.361300  2.000531e-10
d_wh_lag1  0.442161  0.032771  13.492644  0.000000e+00
d_wh_lag2  0.044092  0.057383   0.768367  4.422690e-01
d_wh_lag3  0.119260  0.043272   2.756079  5.849888e-03

First-stage strength
           rsquared  partial.rsquared  shea.rsquared      f.stat  f.pval
d_wh       0.553557          0.358342       0.309343  148.046062     0.0
d_wh_lag1  0.539818          0.348922       0.278144  223.590364     0.0
d_wh_lag2  0.575527          0.377813       0.285632  190.192250     0.0
d_wh_lag3  0.606518          0.385490       0.306715  208.177848     0.0

Cumulative pass-through
cum0    0.242541
cum1    0.684703
cum2    0.728794
cum3    0.848055
dtype: float64


**Interpretation (Regression 3, IV).**
Instrumenting contemporaneous and short-lag wholesale changes with mortality windows yields **large, fast pass-through**:
$\beta_0 \approx 0.26$, $\beta_1 \approx 0.44$, $\beta_3 \approx 0.12$ (lag 2 is small and not significant).
Cumulatively, pass-through reaches **~0.70 by 1 month** and **~0.85 by 3 months**.
The instruments are **very strong**, and **overidentifying restrictions are not rejected** (p $\approx 0.63$–$0.67$).
HAC bandwidth choices change SEs slightly but **do not change inference**.

**Next step.**
Use this IV setup to test **asymmetry** directly.


# 4. Split

In [21]:
# -----------------------
# Build analysis df
# -----------------------
assym_df = (
    df[["date", "price_large", "retail_price", "loss_dth_render"]]
    .rename(columns={"price_large": "wh", "retail_price": "rt", "loss_dth_render": "death"})
    .sort_values("date")
    .assign(death_m=lambda x: x["death"] / 1e6)
)

# ---- Mortality windows (same as before) ----
for lag in range(13):
    assym_df[f"death_m_lag{lag}"] = assym_df["death_m"].shift(lag)

assym_df["D_0_2"]  = assym_df[[f"death_m_lag{l}" for l in range(0, 3)]].sum(axis=1)
assym_df["D_3_6"]  = assym_df[[f"death_m_lag{l}" for l in range(3, 7)]].sum(axis=1)
assym_df["D_7_12"] = assym_df[[f"death_m_lag{l}" for l in range(7, 13)]].sum(axis=1)

for window in ["D_0_2", "D_3_6", "D_7_12"]:
    for lag in range(1, 4):
        assym_df[f"{window}_lag_{lag}"] = assym_df[window].shift(lag)

# ---- Differences ----
assym_df["d_rt"] = assym_df["rt"].diff()
assym_df["d_wh"] = assym_df["wh"].diff()

# -----------------------
# Split wholesale changes into + and - (magnitudes)
#   d_wh_pos >= 0 : increases
#   d_wh_neg >= 0 : decreases (magnitude)
# Interpretation: a $1 *decrease* corresponds to +1 in d_wh_neg
# -----------------------

assym_df["d_wh_pos"] = assym_df["d_wh"].clip(lower=0)   # >=0
assym_df["d_wh_neg"] = assym_df["d_wh"].clip(upper=0)   # <=0  (SIGNED, not magnitude)

K = 9
for side in ["pos", "neg"]:
    for l in range(0, K + 1):
        if l == 0:
            continue
        assym_df[f"d_wh_{side}_lag{l}"] = assym_df[f"d_wh_{side}"].shift(l)


# -----------------------
# Month FE
# -----------------------
assym_df["month"] = assym_df["date"].dt.month
month_dummies = pd.get_dummies(assym_df["month"], prefix="m", drop_first=True).astype(int)
assym_df = pd.concat([assym_df, month_dummies], axis=1)

# Drop NA created by diffs/lags
assym_df = assym_df.dropna().copy()

In [22]:
# -----------------------
# Variable blocks (REG 4: asymmetry)
# Endog: contemporaneous + lags 1-3 for BOTH + and -
# Exog : lags 4..K for BOTH + and - + month FE
# IV   : mortality windows (same)
# -----------------------
endog_cols = (
    ["d_wh_pos", "d_wh_neg"]
    + [f"d_wh_pos_lag{l}" for l in range(1, 4)]
    + [f"d_wh_neg_lag{l}" for l in range(1, 4)]
)

exog_cols = (
    [f"d_wh_pos_lag{l}" for l in range(0, K + 1) if l not in range(0,4)]
    + [f"d_wh_neg_lag{l}" for l in range(0, K + 1) if l not in range(0,4)]
    + list(month_dummies.columns)
)

iv_cols = (
    ["D_0_2", "D_3_6", "D_7_12"]
    + [f"{w}_lag_{l}" for w in ["D_0_2", "D_3_6", "D_7_12"] for l in range(1, 4)]
)

y = assym_df["d_rt"]
endog = assym_df[endog_cols]
exog = sm.add_constant(assym_df[exog_cols])
instruments = assym_df[iv_cols]

# --- Estimate ---
m4 = IV2SLS(y, exog, endog, instruments).fit(
    cov_type="kernel", kernel="bartlett", bandwidth=12
)


In [23]:
# -----------------------
# 1) Main IV estimates (pos/neg only, 0..3)
# -----------------------
key_terms = (
    ["d_wh_pos", "d_wh_neg"]
    + [f"d_wh_pos_lag{l}" for l in range(1, 4)]
    + [f"d_wh_neg_lag{l}" for l in range(1, 4)]
)

key_terms_adj = (
    ["d_wh_pos", "d_wh_neg"]
    + [f"d_wh_pos_lag{l}" for l in range(1, 4)]
    + [f"d_wh_neg_lag{l}" for l in range(1, 4)]
)

b = m4.params.reindex(key_terms_adj)
se = m4.std_errors.reindex(key_terms_adj)
t = b / se
p = 2 * (1 - stats.norm.cdf(np.abs(t)))

out = pd.DataFrame({"beta": b, "se": se, "t": t, "p": p})

print("Main IV estimates (Reg 4)")
print(out)
print()

# -----------------------
# 2) First-stage strength (same terms)
# -----------------------
print("First-stage strength")
print(m4.first_stage.diagnostics.loc[key_terms, ['partial.rsquared', 'f.stat', 'f.pval']])
print()

Main IV estimates (Reg 4)
                   beta        se         t         p
d_wh_pos       0.175169  0.166335  1.053110  0.292290
d_wh_neg       0.279700  0.213021  1.313016  0.189178
d_wh_pos_lag1  0.255998  0.152419  1.679571  0.093041
d_wh_pos_lag2  0.050178  0.277628  0.180740  0.856572
d_wh_pos_lag3  0.453369  0.207058  2.189574  0.028555
d_wh_neg_lag1  0.679265  0.159158  4.267855  0.000020
d_wh_neg_lag2  0.275675  0.182843  1.507714  0.131628
d_wh_neg_lag3 -0.036955  0.141067 -0.261967  0.793347

First-stage strength
               partial.rsquared      f.stat  f.pval
d_wh_pos               0.438771  249.214519     0.0
d_wh_neg               0.359932  135.084196     0.0
d_wh_pos_lag1          0.362016  157.406327     0.0
d_wh_pos_lag2          0.349411  106.559124     0.0
d_wh_pos_lag3          0.388304  196.392924     0.0
d_wh_neg_lag1          0.341046  140.107873     0.0
d_wh_neg_lag2          0.241565  151.233383     0.0
d_wh_neg_lag3          0.363747  116.906954     0.

In [27]:
V = m4.cov
if not isinstance(V, pd.DataFrame):
    V = pd.DataFrame(V, index=m4.params.index, columns=m4.params.index)

rows = []
for h in [0, 1, 2, 3]:
    pos_terms = ["d_wh_pos"] + [f"d_wh_pos_lag{l}" for l in range(1, h+1)]
    neg_terms = ["d_wh_neg"] + [f"d_wh_neg_lag{l}" for l in range(1, h+1)]

    # cumulative COEFFICIENTS (both should be comparable magnitudes in this signed spec)
    cum_pos = float(m4.params.reindex(pos_terms).sum())
    cum_neg = float(m4.params.reindex(neg_terms).sum())

    # H0 no asymmetry: cum_pos - cum_neg = 0
    terms = pos_terms + neg_terms
    w = np.r_[np.ones(len(pos_terms)), -np.ones(len(neg_terms))]  # + for pos, - for neg
    bb = m4.params.reindex(terms).values.astype(float)
    VV = V.loc[terms, terms].values.astype(float)

    diff = float(w @ bb)              # cum_pos - cum_neg
    var  = float(w @ VV @ w)
    chi2 = (diff**2) / var
    pval = 1 - stats.chi2.cdf(chi2, df=1)

    rows.append({"h": h, "cum_pos": cum_pos, "cum_neg": cum_neg,
                 "diff (pos - neg)": diff, "p_no_asym": pval})

cum_tbl = pd.DataFrame(rows).set_index("h")
print("Cumulative pass-through + no-asymmetry test (H0: cum_pos - cum_neg = 0)")
print(cum_tbl)

# -----------------------
# Plot aligned with your template
# -----------------------
berkeley_blue = "#002676"
california_gold = "#FDB515"

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=cum_tbl.index,
    y=cum_tbl["cum_pos"],
    mode="lines+markers",
    name="Cumulative pass-through (wh increase)",
    line=dict(color=berkeley_blue, width=3.5),
    marker=dict(size=8)
))

fig.add_trace(go.Scatter(
    x=cum_tbl.index,
    y=cum_tbl["cum_neg"],
    mode="lines+markers",
    name="Cumulative pass-through (wh decrease)",
    line=dict(color=california_gold, width=3.5),
    marker=dict(size=8)
))

fig.update_layout(
    title="Cumulative pass-through by horizon (Regression 4)",
    xaxis_title="Horizon (months)",
    yaxis_title="Cumulative pass-through",

    font=dict(family="Times New Roman", size=14, color="black"),
    template="simple_white",

    xaxis=dict(
        showgrid=True, gridcolor="lightgray", zeroline=False,
        linecolor="black", linewidth=1, mirror=True,
        ticks="outside", tickwidth=1, tickcolor="black",
        dtick=1
    ),
    yaxis=dict(
        showgrid=True, gridcolor="lightgray", zeroline=False,
        linecolor="black", linewidth=1, mirror=True,
        ticks="outside", tickwidth=1, tickcolor="black"
    ),

    legend=dict(
        x=1.02, y=1, xanchor="left", yanchor="top",
        font=dict(size=15),
        bgcolor="rgba(255,255,255,0)",
        bordercolor="rgba(0,0,0,0)"
    ),

    margin=dict(r=150),
    width=850,
    height=520
)

fig.show()

fig.write_image(PLOT_FOLDER + "Eggs_pass_through.png") #this line saves the plot


Cumulative pass-through + no-asymmetry test (H0: cum_pos - cum_neg = 0)
    cum_pos   cum_neg  diff (pos - neg)  p_no_asym
h                                                 
0  0.175169  0.279700         -0.104530   0.777056
1  0.431167  0.958965         -0.527797   0.360124
2  0.481346  1.234639         -0.753294   0.158186
3  0.934715  1.197685         -0.262969   0.331899


## Takeaways and next steps

Using monthly data, I do **not** find robust evidence of **asymmetric retail adjustment** to wholesale shocks (i.e., no clear “rockets and feathers” pattern) in the specifications explored above. Point estimates sometimes differ across positive vs. negative wholesale changes, but these differences are **not stable across horizons/specifications** and are difficult to distinguish from noise.

This null result has two interpretations:
1. **True symmetry:** retail egg prices may adjust roughly symmetrically to wholesale movements in this period.
2. **Data and frequency limitations:** with a relatively short monthly sample and aggregation from weekly wholesale reports, the analysis may be underpowered to detect asymmetry that operates at higher frequency (e.g., within-week pricing, promotions, or category management).

**Next step.** Replicate the same design using **weekly retail scanner data (Nielsen)** (and potentially regional variation) to increase statistical power and to better align the timing of retail and wholesale movements. If asymmetry exists primarily in adjustment speed rather than long-run pass-through, weekly data should make it substantially easier to detect.
